# 01 - Preparação de Dados

Este notebook carrega e prepara as bases de dados PT-6 e 20NG-6.


In [1]:
import sys
from pathlib import Path

# Adicionar src ao path
project_root = Path().resolve().parent
sys.path.append(str(project_root))

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from src.config import RAW_DATA_DIR, TWENTY_NG_CATEGORIES, PT6_CLASS_COLUMN_CANDIDATES
from src.utils import detect_class_column


## 1. Carregar 20NG-6 (20 Newsgroups - 6 categorias)


In [2]:
# Carregar 20 Newsgroups com 6 categorias selecionadas
newsgroups = fetch_20newsgroups(
    subset='all',
    categories=TWENTY_NG_CATEGORIES,
    remove=('headers', 'footers', 'quotes'),
    shuffle=True,
    random_state=42
)

print(f"Total de documentos: {len(newsgroups.data)}")
print(f"Número de classes: {len(newsgroups.target_names)}")
print(f"Classes: {newsgroups.target_names}")

# Criar DataFrame
df_20ng = pd.DataFrame({
    'text': newsgroups.data,
    'class': newsgroups.target,
    'class_name': [newsgroups.target_names[i] for i in newsgroups.target]
})

print(f"\nDistribuição de classes:")
print(df_20ng['class_name'].value_counts())


Total de documentos: 5906
Número de classes: 6
Classes: ['comp.graphics', 'comp.sys.mac.hardware', 'rec.autos', 'rec.sport.hockey', 'sci.crypt', 'sci.med']

Distribuição de classes:
class_name
rec.sport.hockey         999
sci.crypt                991
rec.autos                990
sci.med                  990
comp.graphics            973
comp.sys.mac.hardware    963
Name: count, dtype: int64


## 2. Carregar PT-6 (CSV)


In [3]:
# Carregar e pré-processar PT-6
pt6_file = RAW_DATA_DIR / "pt6.csv"
pt6_preprocessed_file = RAW_DATA_DIR / "pt6_preprocessed.csv"

# Função para limpar textos (remover NaN e valores vazios)
def clean_texts(df, text_column):
    """Remove linhas com NaN ou valores vazios na coluna de texto."""
    original_len = len(df)
    
    # Criar cópia para não modificar o original
    df_clean = df.copy()
    
    # Converter para string e tratar NaN explicitamente
    df_clean[text_column] = df_clean[text_column].astype(str)
    
    # Remover linhas onde o texto é 'nan' (string), 'None', ou vazio
    mask = (
        (df_clean[text_column] != 'nan') &
        (df_clean[text_column] != 'None') &
        (df_clean[text_column].str.strip() != '') &
        (df_clean[text_column].notna())
    )
    df_clean = df_clean[mask].copy()
    
    # Garantir que são strings válidas e remover espaços extras
    df_clean[text_column] = df_clean[text_column].str.strip()
    
    # Remover qualquer linha que ainda tenha NaN (verificação final)
    df_clean = df_clean[df_clean[text_column].notna()].copy()
    
    cleaned_len = len(df_clean)
    
    if original_len != cleaned_len:
        print(f"⚠️ Removidos {original_len - cleaned_len} documentos inválidos")
    
    # Resetar índice
    df_clean = df_clean.reset_index(drop=True)
    
    # Verificação final: garantir que não há NaN
    nan_count = df_clean[text_column].isna().sum()
    if nan_count > 0:
        print(f"⚠️ ATENÇÃO: Ainda há {nan_count} valores NaN após limpeza!")
        df_clean = df_clean[df_clean[text_column].notna()].reset_index(drop=True)
    
    return df_clean

if pt6_file.exists():
    print(f"📥 Carregando {pt6_file}...")
    df_pt6 = pd.read_csv(pt6_file, encoding='utf-8')
    print(f"   Total inicial: {len(df_pt6)} documentos")
    
    # Detectar coluna de classe dinamicamente
    class_col = detect_class_column(df_pt6, PT6_CLASS_COLUMN_CANDIDATES)
    
    if class_col:
        print(f"   Coluna de classe detectada: {class_col}")
        
        # Detectar coluna de texto (priorizar 'Texto Expandido')
        text_col = 'Texto Expandido' if 'Texto Expandido' in df_pt6.columns else 'Texto Original'
        print(f"   Coluna de texto detectada: {text_col}")
        
        # Limpar dados
        print(f"\\n🧹 Pré-processando dados...")
        df_pt6 = clean_texts(df_pt6, text_col)
        
        # Salvar CSV pré-processado
        df_pt6.to_csv(pt6_preprocessed_file, index=False, encoding='utf-8-sig')
        print(f"   ✅ CSV pré-processado salvo em: {pt6_preprocessed_file}")
        
        print(f"\\n📊 Total após limpeza: {len(df_pt6)} documentos")
        print(f"\\nDistribuição de classes:")
        print(df_pt6[class_col].value_counts())
    else:
        print("ERRO: Não foi possível detectar a coluna de classe!")
        print(f"Colunas disponíveis: {df_pt6.columns.tolist()}")
else:
    print(f"Arquivo não encontrado: {pt6_file}")
    print("Por favor, adicione o arquivo CSV do PT-6 em data/raw/")


📥 Carregando C:\nlp-clustering-benchmark\data\raw\pt6.csv...
   Total inicial: 319 documentos
   Coluna de classe detectada: Categoria
   Coluna de texto detectada: Texto Expandido
\n🧹 Pré-processando dados...
⚠️ Removidos 4 documentos inválidos
   ✅ CSV pré-processado salvo em: C:\nlp-clustering-benchmark\data\raw\pt6_preprocessed.csv
\n📊 Total após limpeza: 315 documentos
\nDistribuição de classes:
Categoria
Turismo                   58
Polícia e Direitos        55
Esportes                  53
Economia                  53
Política                  51
Variedades e Sociedade    45
Name: count, dtype: int64


## 3. Carregar Dados Pré-processados (Opcional)

Se você já executou a célula anterior, pode carregar diretamente o CSV pré-processado.


In [4]:
# Carregar dados pré-processados se necessário
if 'df_pt6' not in locals() or df_pt6 is None:
    pt6_preprocessed_file = RAW_DATA_DIR / "pt6_preprocessed.csv"
    if pt6_preprocessed_file.exists():
        print("📥 Carregando PT-6 pré-processado...")
        df_pt6 = pd.read_csv(pt6_preprocessed_file, encoding='utf-8-sig')
        class_col = detect_class_column(df_pt6, PT6_CLASS_COLUMN_CANDIDATES)
        print(f"   ✅ {len(df_pt6)} documentos carregados")
    else:
        print("⚠️ Arquivo pré-processado não encontrado. Execute a célula anterior primeiro.")

print("\\n" + "=" * 60)
print("RESUMO DOS DADOS")
print("=" * 60)
print(f"\\n📊 20NG-6: {len(df_20ng)} documentos, {df_20ng['class'].nunique()} classes")
if 'df_pt6' in locals() and 'class_col' in locals() and class_col:
    print(f"📊 PT-6: {len(df_pt6)} documentos, {df_pt6[class_col].nunique()} classes")
    print("\\n✅ Dados prontos para geração de embeddings!")
else:
    print("\\n❌ PT-6 não está disponível. Execute a célula anterior primeiro.")


\n============================================================
RESUMO DOS DADOS
\n📊 20NG-6: 5906 documentos, 6 classes
📊 PT-6: 315 documentos, 6 classes
\n✅ Dados prontos para geração de embeddings!
